In [ ]:
!pip install trl

In [1]:
from IPython.display import Image

In [2]:
import os
# os.environ['http_proxy'] = 'http://127.0.0.1:7890'
# os.environ['https_proxy'] = 'http://127.0.0.1:7890'
os.environ['NCCL_P2P_DISABLE'] = '1'
os.environ['NCCL_IB_DISABLE'] = '1'
os.environ['WANDB_DISABLED'] = 'true'

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

## collate_fn (dataloader)

In [4]:
Image(url='https://lukesalamone.github.io/img/torch_collate_fn.png', width=400)

In [5]:
dataset = load_dataset('lucasmccabe-lmi/CodeAlpaca-20k', split='train')

README.md:   0%|          | 0.00/677 [00:00<?, ?B/s]

(…)-00000-of-00001-e270777bb989ac86.parquet:   0%|          | 0.00/3.45M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20022 [00:00<?, ? examples/s]

In [6]:
dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 20022
})

In [7]:
dataset[0]

{'instruction': 'Create a function that takes a specific input and produces a specific output using any mathematical operators. Write corresponding code in Python.',
 'input': '',
 'output': 'def f(x):\n    """\n    Takes a specific input and produces a specific output using any mathematical operators\n    """\n    return x**2 + 3*x'}

## format function

In [8]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"### Question:\n{example['instruction'][i]}\n ### Answer:\n{example['output'][i]}"
        output_texts.append(text)
    return output_texts


In [9]:
output_texts = formatting_prompts_func(dataset[:2])

In [11]:
print(dataset[0], '\n')
print(output_texts[0])

{'instruction': 'Create a function that takes a specific input and produces a specific output using any mathematical operators. Write corresponding code in Python.', 'input': '', 'output': 'def f(x):\n    """\n    Takes a specific input and produces a specific output using any mathematical operators\n    """\n    return x**2 + 3*x'} 

### Question:
Create a function that takes a specific input and produces a specific output using any mathematical operators. Write corresponding code in Python.
 ### Answer:
def f(x):
    """
    Takes a specific input and produces a specific output using any mathematical operators
    """
    return x**2 + 3*x


## DataCollatorForCompletionOnlyLM

In [12]:
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [13]:
tokenizer

GPT2TokenizerFast(name_or_path='facebook/opt-350m', vocab_size=50265, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [15]:
tokenizer.chat_template

In [18]:
response_template = "### Answer:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

## SFTTrainer

In [23]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=SFTConfig(output_dir='/tmp'),
    formatting_func=formatting_prompts_func,
    data_collator=collator,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/Users/bytedance/miniconda3/envs/rl/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [24]:
trainer.train()

  0%|          | 0/7509 [00:00<?, ?it/s]

RuntimeError: MPS backend out of memory (MPS allocated: 30.83 GB, other allocations: 5.18 GB, max allowed: 36.27 GB). Tried to allocate 410.59 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).